In [1]:
import geopandas as gpd
import h3
from shapely.geometry import Point, Polygon, LineString
import folium

In [2]:
build = gpd.read_file("data/F_FAC_BUILDING_서울/F_FAC_BUILDING_11_202210.shp", encoding = 'euc-kr')

In [3]:
build.head(3)

,UFID,BLD_NM,DONG_NM,GRND_FLR,UGRND_FLR,PNU,ARCHAREA,TOTALAREA,PLATAREA,HEIGHT,...,GB_CD,VIOL_BD_YN,GEOIDN,BLDG_PNU,BLDG_PNU_Y,BLD_UNLICE,BD_MGT_SN,SGG_OID,COL_ADM_SE,geometry
0,1991201839054527769900000000,None,None,3.0,0.0,1111017500107040000,57.6,203.13,100.6,9.1,...,0,0,B00100000000T30Z9,None,None,None,1111017500107040000001651,32,11110,"POLYGON ((201844.048 452780.137, 201842.638 45..."
1,1967201343844527773800000000,None,None,3.0,0.0,1111017500100560024,0.0,114.98,0.0,0.0,...,0,0,B00100000000T311B,None,None,None,1111017500100560024002963,34,11110,"POLYGON ((201344.164 452773.285, 201340.604 45..."
2,1962201375194527742000000000,None,None,2.0,0.0,1111017500100560054,0.0,135.20,0.0,0.0,...,0,0,B00100000000T313D,None,None,None,1111017500100560054002866,36,11110,"POLYGON ((201373.004 452770.525, 201372.704 45..."


In [4]:
build_height = build[['GRND_FLR', 'HEIGHT', 'geometry']]
build_height.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,"POLYGON ((201844.048 452780.137, 201842.638 45..."
1,3.0,0.0,"POLYGON ((201344.164 452773.285, 201340.604 45..."
2,2.0,0.0,"POLYGON ((201373.004 452770.525, 201372.704 45..."


In [5]:
# 건물을 Point로 변환하기 위하여 중심점 추출
build_height['geometry'].centroid

0         POINT (201839.038 452776.991)
1         POINT (201343.829 452777.379)
2         POINT (201375.177 452774.201)
3         POINT (201388.462 452771.013)
4         POINT (200468.269 452776.403)
                      ...              
697882    POINT (201821.763 451827.109)
697883    POINT (192517.135 452812.706)
697884    POINT (202329.717 459965.755)
697885    POINT (201343.914 458685.038)
697886    POINT (200626.751 461379.041)
Length: 697887, dtype: geometry

In [6]:
build_point = build_height.copy()
build_point.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,"POLYGON ((201844.048 452780.137, 201842.638 45..."
1,3.0,0.0,"POLYGON ((201344.164 452773.285, 201340.604 45..."
2,2.0,0.0,"POLYGON ((201373.004 452770.525, 201372.704 45..."


In [7]:
build_point['geometry'] = build_point['geometry'].centroid
build_point.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,POINT (201839.038 452776.991)
1,3.0,0.0,POINT (201343.829 452777.379)
2,2.0,0.0,POINT (201375.177 452774.201)


In [8]:
build_point = build_point.to_crs(epsg = 4326)

In [9]:
## Point에서 위경도 좌표 추출
lng = []; lat = []

for i in range(len(build_point)) :
    a = list(build_point['geometry'][i].coords)
    lat.append(a[0][1])
    lng.append(a[0][0])

In [10]:
build_point['lat'] = lat; build_point['lng'] = lng

In [11]:
build_point.head(3)

,GRND_FLR,HEIGHT,geometry,lat,lng
0,3.0,9.1,POINT (127.02161 37.57729),37.577287,127.021607
1,3.0,0.0,POINT (127.01600 37.57729),37.577291,127.016001
2,2.0,0.0,POINT (127.01636 37.57726),37.577262,127.016356


In [12]:
## H3 level 9 index 추가
h3_9 = []

for i in range(len(build_point)) :
    h3_9.append(h3.geo_to_h3(build_point['lat'][i], build_point['lng'][i], 9))

build_point['h3_9'] = h3_9

In [13]:
def to_polygon(l) :
    return Polygon(h3.h3_to_geo_boundary(l, geo_json = True))

In [14]:
## H3 Polygon 데이터 생성
build_h3 = gpd.GeoDataFrame()
build_h3['GRND_FLR'] = build_point['GRND_FLR']
build_h3['HEIGHT'] = build_point['HEIGHT']
build_h3['geometry'] = build_point['h3_9'].apply(to_polygon)

In [15]:
build_h3 = build_h3.set_crs(epsg = 4326)
build_h3.head(3)

,GRND_FLR,HEIGHT,geometry
0,3.0,9.1,"POLYGON ((127.02359 37.57793, 127.02284 37.579..."
1,3.0,0.0,"POLYGON ((127.01816 37.57575, 127.01741 37.577..."
2,2.0,0.0,"POLYGON ((127.01816 37.57575, 127.01741 37.577..."


In [16]:
## H3 grid 내에 포함되는 건물 찾기 위하여 Join
build_join = gpd.sjoin(build_h3, build_point, how = "inner", op = "intersects")
build_join.head(3)

c:\Users\user\Anaconda3\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,GRND_FLR_left,HEIGHT_left,geometry,index_right,GRND_FLR_right,HEIGHT_right,lat,lng,h3_9
0,3.0,9.1,"POLYGON ((127.02359 37.57793, 127.02284 37.579...",223356,1.0,0.0,37.579549,127.021297,8930e1c3263ffff
7,0.0,0.0,"POLYGON ((127.02359 37.57793, 127.02284 37.579...",223356,1.0,0.0,37.579549,127.021297,8930e1c3263ffff
43,0.0,0.0,"POLYGON ((127.02359 37.57793, 127.02284 37.579...",223356,1.0,0.0,37.579549,127.021297,8930e1c3263ffff


In [17]:
build_join2 = build_join.groupby('index_right', as_index = False)[['GRND_FLR_left','HEIGHT_left']].max()

build_group = gpd.GeoDataFrame()
build_group['index'] = build_join2['index_right']
build_group['GRND_FLR'] = build_join2['GRND_FLR_left']
build_group['HEIGHT'] = build_join2['HEIGHT_left']
build_group['geometry'] = build_h3['geometry']
build_group.set_crs(epsg = 4326)

,index,GRND_FLR,HEIGHT,geometry
0,0,14.0,42.40,"POLYGON ((127.02359 37.57793, 127.02284 37.579..."
1,1,8.0,27.85,"POLYGON ((127.01816 37.57575, 127.01741 37.577..."
2,2,8.0,27.85,"POLYGON ((127.01816 37.57575, 127.01741 37.577..."
3,3,8.0,27.85,"POLYGON ((127.01816 37.57575, 127.01741 37.577..."
4,4,8.0,25.50,"POLYGON ((127.00778 37.57705, 127.00703 37.578..."
...,...,...,...,...
697882,697882,33.0,116.00,"POLYGON ((127.02288 37.56946, 127.02214 37.570..."
697883,697883,20.0,53.90,"POLYGON ((126.91868 37.57669, 126.91793 37.578..."
697884,697884,6.0,17.60,"POLYGON ((127.02899 37.64291, 127.02824 37.644..."
697885,697885,6.0,18.50,"POLYGON ((127.01742 37.63007, 127.01667 37.631..."


In [28]:
build_group['HEIGHT'].max()

13418.0

In [33]:
import mapboxgl
from mapboxgl.utils import create_color_stops
from mapboxgl.viz import *

In [26]:
# 서울시청 주소
lat = 37.5662; lng = 126.9787
seoul = [lat, lng]

In [32]:
token = "pk.eyJ1IjoicGdvcWxzIiwiYSI6ImNsOXM2dnU4ejA1aDIzb2s3bjBvOGV4MHEifQ.8MSzkpwd-HyMMSfpVnaciw"
color_breaks = [0, 3355, 6710, 10065, 13420]
color_stops = create_color_stops(color_breaks, colors = "YlOrRd")

In [35]:
m = ChoroplethViz(
    access_token = token,
    data = build_group,
    color_property = "HEIGHT",
    color_stops = color_stops,
    center = seoul,
    zoom = 14
)

m.show()

TypeError: Object of type GeoDataFrame is not JSON serializable